### Business Understanding:

In the fast-paced world of finance, timely access to real-time stock market data is essential for making well-informed decisions, crafting effective strategies, and managing risks. This data wealth offers a unique opportunity to develop predictive models capable of forecasting stock prices, spotting emerging trends, and providing actionable insights to empower investors and traders.

Our proposal introduces a comprehensive data science project designed to harness real-time stock market data for constructing predictive models. By leveraging sophisticated machine learning techniques and advanced data analysis methodologies, our project aims to bridge the gap between raw data and strategic decision-making. We believe that by providing stakeholders in the financial sector with robust predictive tools, we can help them navigate the intricate landscape of the stock market more effectively.

This proposal outlines a structured approach to utilizing real-time stock market data, including data collection, preprocessing, feature engineering, model development, and rigorous evaluation. Through the collaborative efforts of multidisciplinary teams, combined with our commitment to innovation and excellence, we aspire to redefine the boundaries of predictive analytics in finance. Our goal is to create value, manage risks, and foster sustainable growth within the financial ecosystem.



## Import relevant libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series

## Load Data

In [2]:
stocks_df = pd.read_excel("Data/update_stock_data.xlsx", index_col = 0)
stocks_df.head()

,close,high,low,open,priceDate,symbol,volume,id,key,subkey,...,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent,Name
2022-01-03,182.01,182.88,177.71,177.83,2022-01-03,AAPL,104701220.0,HISTORICAL_PRICES,AAPL,NaN,...,104701220,175.358163,179.480061,180.337968,175.239831,104701220.0,2022-01-03,4.44,0.0250,Apple Inc.
2022-01-04,179.70,182.94,179.12,182.63,2022-01-04,AAPL,99310438.0,HISTORICAL_PRICES,AAPL,NaN,...,99310438,180.091443,177.202170,180.397134,176.630232,99310438.0,2022-01-04,-2.31,-0.0127,Apple Inc.
2022-01-05,174.92,180.17,174.64,179.61,2022-01-05,AAPL,94537602.0,HISTORICAL_PRICES,AAPL,NaN,...,94537602,177.113421,172.488612,177.665637,172.212504,94537602.0,2022-01-05,-4.78,-0.0266,Apple Inc.
2022-01-06,172.00,175.30,171.64,172.70,2022-01-06,AAPL,96903955.0,HISTORICAL_PRICES,AAPL,NaN,...,96903955,170.299470,169.609200,172.863330,169.254204,96903955.0,2022-01-06,-2.92,-0.0167,Apple Inc.
2022-01-07,172.17,174.14,171.03,172.89,2022-01-07,AAPL,86709147.0,HISTORICAL_PRICES,AAPL,NaN,...,86709147,170.486829,169.776837,171.719454,168.652683,86709147.0,2022-01-07,0.17,0.0010,Apple Inc.


In [3]:
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10509 entries, 2022-01-03 to 2022-12-30
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   close                 10509 non-null  float64       
 1   high                  10509 non-null  float64       
 2   low                   10509 non-null  float64       
 3   open                  10509 non-null  float64       
 4   priceDate             10509 non-null  datetime64[ns]
 5   symbol                10509 non-null  object        
 6   volume                10509 non-null  float64       
 7   id                    10509 non-null  object        
 8   key                   10509 non-null  object        
 9   subkey                0 non-null      float64       
 10  updated               10509 non-null  float64       
 11  changeOverTime        10509 non-null  float64       
 12  marketChangeOverTime  10509 non-null  float64       
 13 

In [4]:
stocks_df.describe()

,close,high,low,open,priceDate,volume,subkey,updated,changeOverTime,marketChangeOverTime,...,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent
count,10509.000000,10509.000000,10509.000000,10509.000000,10509,1.050900e+04,0.0,1.050900e+04,10509.000000,10509.000000,...,10509.000000,1.050900e+04,10509.000000,10509.000000,10509.000000,10509.000000,1.050900e+04,10509,10509.000000,10509.000000
mean,223.485346,227.252650,219.847448,223.738443,2022-06-30 12:05:32.971738624,2.922589e+07,NaN,1.712387e+12,0.850805,0.850805,...,327.361217,2.455677e+07,161.398618,161.268858,163.939047,158.671194,2.922589e+07,2022-06-30 12:05:32.971738624,-0.925026,-0.001275
min,25.040000,25.480000,24.590000,24.725000,2022-01-03 00:00:00,1.197767e+06,NaN,1.659625e+12,-0.955650,-0.955650,...,24.590000,7.458600e+05,23.770615,24.073456,24.496472,23.640826,1.197767e+06,2022-01-03 00:00:00,-2322.210000,-0.949000
25%,96.310000,98.230000,95.030000,96.520000,2022-03-31 00:00:00,7.624161e+06,NaN,1.713002e+12,-0.111187,-0.111187,...,95.030000,6.448739e+06,83.120000,83.139342,84.820000,81.765450,7.624161e+06,2022-03-31 00:00:00,-2.050000,-0.015000
50%,147.630000,149.675000,145.522505,147.470000,2022-06-29 00:00:00,1.893468e+07,NaN,1.713002e+12,0.233579,0.233579,...,154.950000,1.541502e+07,140.900500,140.489334,142.607750,138.797000,1.893468e+07,2022-06-29 00:00:00,-0.130000,-0.001300
75%,231.840000,236.600000,226.420000,231.300000,2022-09-29 00:00:00,3.991910e+07,NaN,1.713002e+12,0.483837,0.483837,...,265.610000,3.272002e+07,220.945000,221.420000,225.750000,216.350000,3.991910e+07,2022-09-29 00:00:00,1.660000,0.012500
max,3408.090000,3428.000000,3357.370000,3408.760000,2022-12-30 00:00:00,2.732081e+08,NaN,1.713003e+12,21.506579,21.506579,...,3357.370000,2.231334e+08,605.610000,597.370000,609.990000,590.560000,2.732081e+08,2022-12-30 00:00:00,375.880000,0.367600
std,400.097648,407.336922,393.242690,401.052866,NaN,2.963628e+07,NaN,4.184407e+09,3.022659,3.022659,...,594.171843,2.628159e+07,105.304724,105.077810,107.072277,103.192552,2.963628e+07,NaN,35.270362,0.030460


In [5]:
stocks_df.shape

(10509, 27)

In [6]:
stocks_df.dtypes

close                          float64
high                           float64
low                            float64
open                           float64
priceDate               datetime64[ns]
symbol                          object
volume                         float64
id                              object
key                             object
subkey                         float64
updated                        float64
changeOverTime                 float64
marketChangeOverTime           float64
uOpen                          float64
uClose                         float64
uHigh                          float64
uLow                           float64
uVolume                          int64
fOpen                          float64
fClose                         float64
fHigh                          float64
fLow                           float64
fVolume                        float64
label                   datetime64[ns]
change                         float64
changePercent            

## Data Cleaning

In [7]:
stocks_df.dtypes

close                          float64
high                           float64
low                            float64
open                           float64
priceDate               datetime64[ns]
symbol                          object
volume                         float64
id                              object
key                             object
subkey                         float64
updated                        float64
changeOverTime                 float64
marketChangeOverTime           float64
uOpen                          float64
uClose                         float64
uHigh                          float64
uLow                           float64
uVolume                          int64
fOpen                          float64
fClose                         float64
fHigh                          float64
fLow                           float64
fVolume                        float64
label                   datetime64[ns]
change                         float64
changePercent            

In [8]:
def missing_values(data):
    # identify the total missing values per column
    # sort in order
    miss = data.isnull().sum().sort_values(ascending = False)

    # calculate percentage of the missing values
    percentage_miss = (data.isnull().sum() / len(data)).sort_values(ascending = False)

    # store in a dataframe
    missing = pd.DataFrame({"Missing Values": miss, "Percentage": percentage_miss}).reset_index()

    # remove values that are missing
    missing.drop(missing[missing["Percentage"] == 0].index, inplace = True)

    return missing

In [9]:
missing_data = missing_values(stocks_df)
missing_data                                                

,index,Missing Values,Percentage
0,subkey,10509,1.0


In [10]:
stocks_df = stocks_df.drop('subkey', axis = 1)
stocks_df.isna().sum()

close                   0
high                    0
low                     0
open                    0
priceDate               0
symbol                  0
volume                  0
id                      0
key                     0
updated                 0
changeOverTime          0
marketChangeOverTime    0
uOpen                   0
uClose                  0
uHigh                   0
uLow                    0
uVolume                 0
fOpen                   0
fClose                  0
fHigh                   0
fLow                    0
fVolume                 0
label                   0
change                  0
changePercent           0
Name                    0
dtype: int64

In [11]:
## set priceDate  as index
stocks_df.set_index('priceDate', inplace = True)

In [12]:
stocks_df.index

DatetimeIndex(['2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06',
               '2022-01-07', '2022-01-10', '2022-01-11', '2022-01-12',
               '2022-01-13', '2022-01-14',
               ...
               '2022-12-16', '2022-12-19', '2022-12-20', '2022-12-21',
               '2022-12-22', '2022-12-23', '2022-12-27', '2022-12-28',
               '2022-12-29', '2022-12-30'],
              dtype='datetime64[ns]', name='priceDate', length=10509, freq=None)

## Exploratory Data Analysis

In [13]:
# Check the data type of the 'priceDate' column
print(stocks_df.index.dtype)

# Verify if there are any NaN or missing values in the 'priceDate' column
print(stocks_df.index.isna().sum())

# Display the first few rows of the DataFrame to inspect the 'priceDate' column
print(stocks_df.head())


datetime64[ns]
0
             close    high     low    open symbol       volume  \
priceDate                                                        
2022-01-03  182.01  182.88  177.71  177.83   AAPL  104701220.0   
2022-01-04  179.70  182.94  179.12  182.63   AAPL   99310438.0   
2022-01-05  174.92  180.17  174.64  179.61   AAPL   94537602.0   
2022-01-06  172.00  175.30  171.64  172.70   AAPL   96903955.0   
2022-01-07  172.17  174.14  171.03  172.89   AAPL   86709147.0   

                           id   key       updated  changeOverTime  ...  \
priceDate                                                          ...   
2022-01-03  HISTORICAL_PRICES  AAPL  1.713001e+12       -0.086433  ...   
2022-01-04  HISTORICAL_PRICES  AAPL  1.713001e+12       -0.098027  ...   
2022-01-05  HISTORICAL_PRICES  AAPL  1.713001e+12       -0.122020  ...   
2022-01-06  HISTORICAL_PRICES  AAPL  1.713001e+12       -0.136676  ...   
2022-01-07  HISTORICAL_PRICES  AAPL  1.713001e+12       -0.135823  ...   

 

In [14]:
# Check the data type of the index
print(stocks_df.index.dtype)

# If the index is in datetime format, proceed with datetime slicing
if isinstance(stocks_df.index, pd.DatetimeIndex):
    # Slice the DataFrame based on datetime index
    sliced_df = stocks_df.loc['2000-01-01':]
else:
    # If the index is not datetime, you may need to convert it or slice differently based on its format
    # For example, if the index is in string format representing dates, you can do:
    # sliced_df = stocks_df.loc['2000-01-01':]

    # Plot the DataFrame directly if no further slicing is needed
    stocks_df.plot(figsize=(30, 15))


datetime64[ns]


KeyError: 'Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is not allowed.'